# Overview

In this notebook we'll value FTR option prices using the limited data provided in the spreadsheet here:

Ultimately, we would like to compare the neural net created here with one created on simulated data using assumption regarding the behavior of electricity prices (particular SDE formulas).

In [1]:
import pandas as pd

In [10]:
option_data = pd.read_csv("./csvs/option_data.csv")
option_path_data = pd.read_csv("./csvs/option_path_data.csv")
lmp_data = pd.read_csv("./csvs/lmp_data.csv")

In [11]:
option_data = option_data.drop(option_data.columns[0], axis=1)
option_path_data = option_path_data.drop(option_path_data.columns[0], axis=1)
lmp_data = lmp_data.drop(lmp_data.columns[0], axis=1)

Let's add source and sink LMP prices from the lmp_data dataframe to the option data dataframe.

In [21]:
lmp_data1 = lmp_data.rename(columns={"PNODEID":"Source PNODEID"})
model_input_data = lmp_data1.drop("Node", axis=1).merge(option_data, on="Source PNODEID")
model_input_data = model_input_data.rename(columns={"LMP_24H":"LMP day source", "LMP.1_On Peak": "LMP on source",  "LMP.2_Off Peak": "LMP off source",  "LMP.3_Wknd On Peak": "LMP wknd on source", "LMP.4_Daily Off Peak": "LMP daily off source"})

In [22]:
lmp_data2 = lmp_data.rename(columns={"PNODEID":"Sink PNODEID"})
model_input_data = lmp_data2.drop("Node", axis=1).merge(model_input_data, on="Sink PNODEID")
model_input_data = model_input_data.rename(columns={"LMP_24H":"LMP day sink", "LMP.1_On Peak": "LMP on sink",  "LMP.2_Off Peak": "LMP off sink",  "LMP.3_Wknd On Peak": "LMP wknd on sink", "LMP.4_Daily Off Peak": "LMP daily off sink"})

In [23]:
model_input_data

,Sink PNODEID,Period Type,LMP day sink,LMP on sink,LMP off sink,LMP wknd on sink,LMP daily off sink,Source PNODEID,Period Type_x,LMP day source,...,Class Type,Period Type_y,Participant,Source Node,Sink Node,Trade Type,Hedge Type,Cleared MW,Obligation MCP,Option MCP
0,4.024388e+07,All,28185.26,18784.16,9401.09,4966.95,4434.14,34508503.0,All,27695.86,...,24H,All,FEELLC,DAY,09GRNVIL13.8 KV GT2,Buy,Option,6.0,489.40,1201.68
1,4.024388e+07,All,28185.26,18784.16,9401.09,4966.95,4434.14,34508503.0,All,27695.86,...,24H,All,BJE,DAY,09GRNVIL13.8 KV GT2,Buy,Option,3.8,489.40,1201.68
2,4.024388e+07,All,28185.26,18784.16,9401.09,4966.95,4434.14,34508503.0,All,27695.86,...,24H,All,VAEM10,DAY,09GRNVIL13.8 KV GT2,Buy,Option,3.0,489.40,1201.68
3,4.024388e+07,All,28185.26,18784.16,9401.09,4966.95,4434.14,34508503.0,All,27695.86,...,24H,All,BIGBND,DAY,09GRNVIL13.8 KV GT2,Buy,Option,7.0,489.40,1201.68
4,4.024388e+07,All,28185.26,18784.16,9401.09,4966.95,4434.14,34508503.0,All,27695.86,...,OnPeak,All,SENECA,DAY,09GRNVIL13.8 KV GT2,Buy,Option,5.6,178.55,690.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8418,2.156111e+09,All,-6845.12,-4568.55,-2276.58,-1337.26,-939.32,51295.0,All,-3208.67,...,OnPeak,All,DYNECA,JCPL,YARDVILL13 KV YARDVISP,Buy,Option,0.3,-2214.41,1201.06
8419,2.156111e+09,All,-6845.12,-4568.55,-2276.58,-1337.26,-939.32,51295.0,All,-3208.67,...,OnPeak,All,JANTR2,JCPL,YARDVILL13 KV YARDVISP,Buy,Option,1.6,-2214.41,1201.06
8420,2.156111e+09,All,-6845.12,-4568.55,-2276.58,-1337.26,-939.32,51295.0,All,-3208.67,...,OnPeak,All,SENECA,JCPL,YARDVILL13 KV YARDVISP,Buy,Option,3.8,-2214.41,1201.06
8421,2.156111e+09,All,-6845.12,-4568.55,-2276.58,-1337.26,-939.32,51295.0,All,-3208.67,...,DailyOffPeak,All,FEELLC,JCPL,YARDVILL13 KV YARDVISP,Buy,Option,6.0,-650.05,338.26


# Prepare Data

We'll prepare the variables in the `model_input_data` dataframe for the neural network.  The target variable is `Option MCP`.